In [1]:
import matplotlib.pyplot as plt
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Dataset
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import torchvision.utils as vutils
import random
import os
import shutil
import torch
import torch.nn as nn
from torch import optim
from torch.autograd import Variable
import numpy as np
import os

In [2]:
class make_dataset(Dataset):

    def __init__(self, data,transform=None):
        self.patches = data['X']
        self.labels = data['Y']
        self.transform = transform



    def __len__(self):
        return self.labels.shape[0]

    def __getitem__(self, idx):
    
        return self.transform(self.patches[idx]),self.labels[idx]

In [3]:
# Set Defaults
num_epochs = 10 
batch_size = 16
num_classes = 10 # number of classes for dataset
lr = 0.0002 
b1 = 0.5 # adam: decay of first order momentum of gradient
b2 = 0.999 # adam: decay of first order momentum of gradient
n_cpu = 8 # number of cpu threads to use during batch generation
latent_dim = 100 # dimensionality of the latent space
img_size = 256 # size of each image dimension
channels = 1 # number of output image channels
sample_interval = 400 # interval between image sampling

In [4]:
# Set cuda
if torch.cuda.is_available():
    cuda = True 
else:
    cuda = False

In [5]:
def weights_init_normal(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        m.weight.data.normal_(0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        m.weight.data.normal_(1.0, 0.02)
        m.bias.data.zero_()

In [6]:
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.use_gpu = cuda
        drop_rate = 0.20
        num_classes = 2
        input_channels = 3
        ndf = 16
        
        self.conv1 = nn.Sequential(
            nn.Conv2d(in_channels=input_channels, out_channels=ndf, kernel_size=3, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(ndf),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Dropout2d(drop_rate),
        )
        self.conv2 = nn.Sequential(
            nn.Conv2d(in_channels=ndf, out_channels=ndf*2, kernel_size=3, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(ndf*2),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Dropout2d(drop_rate),
        )
        self.conv3 = nn.Sequential(
            nn.Conv2d(in_channels=ndf*2, out_channels=ndf*4, kernel_size=3, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(ndf*4),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Dropout2d(drop_rate),
        )
        self.conv4 = nn.Sequential(
            nn.Conv2d(in_channels=ndf*4, out_channels=ndf*8, kernel_size=3, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(ndf*8),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Dropout2d(drop_rate),
        )
        
        # The height and width of downsampled image
        ds_size = img_size // 2**4
        
        # Output layer
        self.linear1 = nn.Sequential(
            nn.Linear(in_features=128*ds_size**2, out_features=1),
            nn.Sigmoid()
        )

    def forward(self, x):
        #print("---------------------------------------")
        #print("input size = ", x.size())
        x = self.conv1(x)
        #print("Conv1 output = ", x.size())
        x = self.conv2(x)
        #print("Conv2 output = ", x.size())
        x = self.conv3(x)
        #print("Conv3 output = ", x.size())
        x = self.conv4(x)
        #print("Conv4 output = ", x.size())
        
        # Flatten the features to give as input to the fc linear classification layer
        out = x.view(x.shape[0], -1)
        #print("Out flatten shape = ", out.size())
        out = self.linear1(out)
        #print("Linear output = ", out.size())
        #print("---------------------------------------")
        
        return out

In [7]:
# Initialize loss and model
loss = nn.BCELoss()
model = Model()

if cuda:
    loss.cuda()
    model = nn.DataParallel(model)
    model.cuda()

In [8]:
# Initialize weights
model.apply(weights_init_normal)
optimizer = optimizer = torch.optim.Adam(model.parameters(), lr=lr,betas=(b1, b2))

In [9]:
# Load Data
dataset_path = '../dataset/training_data.npz'
data = np.load(dataset_path)
transformations = transforms.Compose([transforms.ToTensor(),transforms.Normalize((.5, .5, .5), (.5, .5, .5))])
training_data = make_dataset(data,transformations)
train_loader = DataLoader(training_data, batch_size=batch_size, shuffle=True)

In [10]:
def train(optimizer,criterion,dataloader,model):
    model.train()
    epoch_loss = 0
    for i, (imgs, labels) in enumerate(dataloader):
            imgs = Variable(imgs.float())
            
            labels = Variable(labels.float())
            optimizer.zero_grad()
            outputs = model(imgs)
            loss = criterion(outputs, labels)
            epoch_loss+=loss.item()
            loss.backward()
            optimizer.step()
            #print(i)
    
    return np.mean(epoch_loss)

    

In [11]:
for epoch in range(num_epochs):
    #train model
    epoch_loss = train(optimizer,loss,train_loader,model)
    #validation
    
    #test
    
    print('Epoch [%d], Loss: %.4f'% (epoch + 1,epoch_loss))
    
        
        

/home/ubuntu/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/torch/nn/functional.py:1594: UserWarning: Using a target size (torch.Size([16])) that is different to the input size (torch.Size([16, 1])) is deprecated. Please ensure they have the same size.
  "Please ensure they have the same size.".format(target.size(), input.size()))
/home/ubuntu/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/torch/nn/functional.py:1594: UserWarning: Using a target size (torch.Size([9])) that is different to the input size (torch.Size([9, 1])) is deprecated. Please ensure they have the same size.
  "Please ensure they have the same size.".format(target.size(), input.size()))


Epoch [1], Loss: 4.7806
Epoch [2], Loss: 3.3269
Epoch [3], Loss: 2.8183
Epoch [4], Loss: 1.5490
Epoch [5], Loss: 1.1461
Epoch [6], Loss: 0.7695
Epoch [7], Loss: 0.6078
Epoch [8], Loss: 0.3612
Epoch [9], Loss: 0.1618
Epoch [10], Loss: 0.3235
